In [1]:
!pip install --upgrade yahoo_fin

In [2]:
import pandas as pd
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from yahoo_fin.stock_info import get_data
from flask import Blueprint
import requests
import json
from flask_sqlalchemy import SQLAlchemy
from flask import Flask, render_template
from os import path
import csv
import pprint

import pandas as pd
Finance_db = "database.db" ## defining the name of the SQLite database
db = SQLAlchemy()



In [3]:
## Import NYSE stock from CSV
NYSE_stock_df = pd.read_csv('/Users/ellandalla/Desktop/website Project/Data/NYSE_stocks.csv')

In [4]:
NYSE_stock_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$125.05,-2.66,-2.08%,3.693674e+10,United States,1999.0,1742578,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$34.50,0.57,1.68%,6.156511e+09,United States,2016.0,4243329,Industrials,Aluminum
2,AAIC,Arlington Asset Investment Corp Class A (new),$4.43,-0.12,-2.64%,1.256368e+08,United States,NaN,289165,Real Estate,Real Estate Investment Trusts
3,AAIN,Arlington Asset Investment Corp 6.000% Senior ...,$23.71,0.00,0.00%,6.724234e+08,United States,NaN,10,Real Estate,Real Estate Investment Trusts
4,AAN,Aarons Holdings Company Inc. Common Stock,$13.15,-1.08,-7.59%,4.059274e+08,United States,2020.0,294314,Consumer Discretionary,Diversified Commercial Services


In [ ]:
stock_df_results = pd.DataFrame()
# Use a for loop to fetch historical data from YFinance
for symbol in NYSE_stock_df['Symbol']:
    try:
        data = get_data(symbol, start_date="2014-01-01", end_date="2023-07-31", index_as_date = True, interval="1mo")
        if not data.empty:
            stock_df_results = pd.concat([stock_df_results, data])
        else:
            print(f"No data available for {symbol}")
        # Process the data here
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        
      

Error fetching data for ABC: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for APE: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for ARNC: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}


In [ ]:
stock_df_results.iloc[:5]

In [ ]:
stock_df_results.iloc[:-5]

In [ ]:
# merge the stock_df_results with stock_df to get the sector
NYSE_final_df = pd.merge(NYSE_stock_df[['Symbol', 'Sector']], stock_df_results.reset_index(), 
                            left_on="Symbol", right_on="ticker")
## Group stock by sector and by price
grouped = NYSE_final_df.groupby(['Sector', 'index']).sum()
grouped_data = pd.DataFrame(grouped)

te = grouped_data.index.get_level_values(index)
#grouped_data
#NYSE_fin
grouped_data.head()
drop_list=['high', 'low', 'adjclose', 'open' ]
for item in drop_list:
    grouped_data = grouped_data.drop(item, axis=1)
grouped_data.head()

In [ ]:
grouped_data.head()

In [ ]:
grouped_data.reset_index(inplace=True)

In [ ]:
grouped_data.head()

In [ ]:
#pivot_df_Price = pd.pivot_table(grouped_data, index = ['index'], columns = ['Sector'], values = 'Change in Price').reset_index()

In [ ]:
#pivot_df_Price

In [ ]:
CAGR_data = {}
for sector in pivot_df_Price.columns[1:]:
    # initiaitlize the sector names in dictionary
    CAGR_data[sector] = {}
    # Add the keys in the deictionary for the sector key 
    CAGR_data[sector]['X'] = pivot_df_Price['index'].astype(str).to_list()
    CAGR_data[sector]['Y'] = pivot_df_Price[sector].to_list()
print(type(CAGR_data))

In [ ]:
## converting to json file
file_path = "CAGR_data.json"
with open(file_path, "w") as json_file:
    json.dump(CAGR_data, json_file)

In [ ]:
# To capture CAGR Volume 
pivot_df_Volume = pd.pivot_table(grouped_data, index = ['index'], columns = ['Sector'], values = 'Change in Volume').reset_index()

In [ ]:

pivot_df_Volume

In [ ]:
CAGR_Volume = {}
for sector in pivot_df_Volume.columns[1:]:
    # initiaitlize the sector names in dictionary
    CAGR_Volume[sector] = {}
    # Add the keys in the deictionary for the sector key 
    CAGR_Volume[sector]['X'] = pivot_df_Volume['index'].astype(str).to_list()
    CAGR_Volume[sector]['Y'] = pivot_df_Volume[sector].to_list()

In [ ]:
file_path = "CAGR_volume.json"
with open(file_path, "w") as json_file:
    json.dump(CAGR_Volume, json_file)

In [ ]:
CAGR_Volume['Finance']['Y']

In [ ]:
## To capture regular volume per sector
pivot_df_Sector_volume = pd.pivot_table(grouped_data,columns = ['Sector'], values ='volume').reset_index()


pivot_df_Sector_volume

In [ ]:
Sector_Volume = {}
Sector_Volume['X'] = pivot_df_Sector_volume[sector].to_list()
Sector_Volume['Y'] = pivot_df_Sector_volume['index'].astype(str).to_list()
    

In [ ]:
file1 = open('Sector_Volume.txt', 'w')
file1.write(str(Sector_Volume))
file1.close()